# Company1 - ETL Process

This notebook is organized in the following sections:
* [Step 0 - Preliminary: Viewing the data](#0)
* [Step 1 - Checking for duplicates](#1)
* [Step 2 - Checking for missing values](#2)
* [Step 3 - Imputing/dropping missing values](#3)
* [Step 4 - Ensuring Correct Datatypes](#4)
* [Step 5 - Preparation for merging](#5)


<a id='0'></a>
## Step 0 - Preliminary: Viewing the data

In [2]:
import pandas as pd

In [3]:
# Load the share prices csv

prices = pd.read_csv("data/us-shareprices-daily.csv",delimiter=';')

In [4]:
prices.tail()

,Ticker,SimFinId,Date,Open,High,Low,Close,Adj. Close,Volume,Dividend,Shares Outstanding
5767354,ZYXI,171401,2024-02-26,13.04,13.04,12.67,12.82,12.82,335055,NaN,36435000.0
5767355,ZYXI,171401,2024-02-27,12.83,13.77,12.83,13.74,13.74,395525,NaN,36435000.0
5767356,ZYXI,171401,2024-02-28,13.63,13.70,13.38,13.49,13.49,290887,NaN,32170182.0
5767357,ZYXI,171401,2024-02-29,13.51,13.57,13.28,13.56,13.56,232534,NaN,32170182.0
5767358,ZYXI,171401,2024-03-01,12.05,13.43,12.00,12.30,12.30,1216112,NaN,32170182.0


In [5]:
# Load the companies csv 
companies = pd.read_csv("data/us-companies.csv",delimiter=';')

In [6]:
companies.tail()

,Ticker,SimFinId,Company Name,IndustryId,ISIN,End of financial year (month),Number Employees,Business Summary,Market,CIK,Main Currency
6065,ZWS,17663788,Zurn Elkay Water Solutions Corporation,100001.0,US98983L1089,12.0,2700.0,Zurn Elkay Water Solutions Corporation designs...,us,1439288.0,USD
6066,ZY,1243193,Zymergen Inc.,106002.0,US98985X1000,12.0,758.0,Zymergen is a biofacturing company using biolo...,us,1645842.0,USD
6067,ZYME,17663790,Zymeworks Inc.,106002.0,CA98985W1023,12.0,291.0,"Zymeworks Inc., a clinical-stage biopharmaceut...",us,1403752.0,USD
6068,ZYNE,901704,"Zynerba Pharmaceuticals, Inc.",106002.0,US98986X1090,12.0,25.0,Zynerba Pharmaceuticals Inc together with its ...,us,1621443.0,USD
6069,ZYXI,171401,ZYNEX INC,106004.0,US98986M1036,12.0,768.0,"Zynex, Inc. engages in the design, manufacture...",us,846475.0,USD


Selecting a company:

In [7]:
unique_companies = list(companies["Company Name"].unique())
unique_companies

[nan,
 'Atlanta Braves Holdings, Inc.',
 'BurgerFi International, Inc.',
 'AGILENT TECHNOLOGIES INC',
 'Li Auto Inc.',
 'Alcoa Corp',
 'Ares Acquisition Corporation',
 'Armada Acquisition Corp. I',
 'AAC Holdings, Inc.',
 'America Great Health',
 'Arlington Asset Investment Corp.',
 'American Airlines Group Inc.',
 'Altisource Asset Management Corp',
 'ATLANTIC AMERICAN CORP',
 "Aaron's Company, Inc.",
 'APPLIED OPTOELECTRONICS, INC.',
 'AAON INC',
 'ADVANCE AUTO PARTS INC',
 'APPLE INC',
 'American Assets Trust, Inc.',
 'Agape ATP Corporation',
 'Ascend Wellness Holdings, Inc.',
 'ATLAS AIR WORLDWIDE HOLDINGS INC',
 'AllianceBernstein Holding L.P.',
 'ABAXIS INC',
 'AbbVie Inc.',
 'Ameris Bancorp',
 'CAMBIUM LEARNING GROUP, INC.',
 'AbCellera Biologics Inc.',
 'ABEONA THERAPEUTICS INC.',
 'ASBURY AUTOMOTIVE GROUP INC',
 'ARCA biopharma, Inc.',
 'ABM INDUSTRIES INC /DE/',
 'AMERICAN BIO MEDICA CORP',
 'Abiomed Inc',
 'American Battery Technology Company',
 'Airbnb, Inc.',
 'Acumen Phar

In [8]:
company_bruker = companies[companies["SimFinId"] == 1253240]
company_bruker

,Ticker,SimFinId,Company Name,IndustryId,ISIN,End of financial year (month),Number Employees,Business Summary,Market,CIK,Main Currency
930,BRKR,1253240,BRUKER CORP,106001.0,US1167941087,12.0,7400.0,Bruker is enabling scientists to make breakthr...,us,1109354.0,USD


In [9]:
prices_bruker = prices[prices["SimFinId"] == 1253240]
prices_bruker

,Ticker,SimFinId,Date,Open,High,Low,Close,Adj. Close,Volume,Dividend,Shares Outstanding
812641,BRKR,1253240,2019-04-01,38.76,39.01,38.47,38.87,38.20,761845,NaN,156814676.0
812642,BRKR,1253240,2019-04-02,38.94,39.01,38.39,38.53,37.86,666801,NaN,156814676.0
812643,BRKR,1253240,2019-04-03,38.74,39.24,38.63,39.01,38.33,2647355,NaN,156814676.0
812644,BRKR,1253240,2019-04-04,38.95,39.01,37.82,38.32,37.66,503730,NaN,156814676.0
812645,BRKR,1253240,2019-04-05,38.41,39.23,38.30,38.41,37.74,509516,NaN,156814676.0
...,...,...,...,...,...,...,...,...,...,...,...
813875,BRKR,1253240,2024-02-26,84.34,84.88,83.21,83.60,83.35,788986,NaN,137671143.0
813876,BRKR,1253240,2024-02-27,83.60,84.29,83.05,83.70,83.45,559922,NaN,137671143.0
813877,BRKR,1253240,2024-02-28,84.18,87.25,84.13,86.48,86.23,1433649,NaN,137671143.0
813878,BRKR,1253240,2024-02-29,86.90,88.92,85.86,86.54,86.33,2687003,0.05,137671143.0


<a id='1'></a>
## Step 1 - Checking for duplicates

In [10]:
#Check for share prices

prices.duplicated().any()

False

In [11]:
# Check for companies 

companies.duplicated().any()

False

Conclusion: There are no duplicate values.

<a id='2'></a>
## Step 2 - Checking for missing values

**Checking full data sets first:**

In [12]:
# Check missing values for share prices

prices.isna().sum() / len(prices)

Ticker                0.000000
SimFinId              0.000000
Date                  0.000000
Open                  0.000000
High                  0.000000
Low                   0.000000
Close                 0.000000
Adj. Close            0.000000
Volume                0.000000
Dividend              0.993940
Shares Outstanding    0.089509
dtype: float64

In [13]:
# Check missing values for companies

companies.isna().sum() / len(companies)

Ticker                           0.012521
SimFinId                         0.000000
Company Name                     0.012191
IndustryId                       0.047117
ISIN                             0.178254
End of financial year (month)    0.012026
Number Employees                 0.124053
Business Summary                 0.051400
Market                           0.000000
CIK                              0.001977
Main Currency                    0.000000
dtype: float64

**Checking in specific company:**

In [15]:
prices_bruker.isna().sum() / len(prices_bruker)

Ticker                0.000000
SimFinId              0.000000
Date                  0.000000
Open                  0.000000
High                  0.000000
Low                   0.000000
Close                 0.000000
Adj. Close            0.000000
Volume                0.000000
Dividend              0.983858
Shares Outstanding    0.000000
dtype: float64

In [20]:
company_bruker.isna().sum() / len(company_bruker)

Ticker                           0.0
SimFinId                         0.0
Company Name                     0.0
IndustryId                       0.0
ISIN                             0.0
End of financial year (month)    0.0
Number Employees                 0.0
Business Summary                 0.0
Market                           0.0
CIK                              0.0
Main Currency                    0.0
dtype: float64

Conclusion: The only missing values are dividends in the prices_bruker df, which are not paid out all the time. We can assume they equal 0 when they are missing.

<a id='3'></a>
## Step 3 - Imputing/dropping missing values

In [16]:
prices_bruker["Dividend"] = prices_bruker["Dividend"].fillna(0)

/var/folders/9n/822nl7nj22lg267k4mhz7kw40000gn/T/ipykernel_86460/902796419.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_bruker["Dividend"] = prices_bruker["Dividend"].fillna(0)


In [17]:
prices_bruker.Dividend.unique()

array([0.  , 0.04, 0.05])

In [18]:
prices_bruker.isna().sum() / len(prices_bruker)

Ticker                0.0
SimFinId              0.0
Date                  0.0
Open                  0.0
High                  0.0
Low                   0.0
Close                 0.0
Adj. Close            0.0
Volume                0.0
Dividend              0.0
Shares Outstanding    0.0
dtype: float64

Conclusion: No missing values anymore.

<a id='4'></a>
## Step 4 - Ensuring correct datatypes

In [21]:
prices_bruker.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1239 entries, 812641 to 813879
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Ticker              1239 non-null   object 
 1   SimFinId            1239 non-null   int64  
 2   Date                1239 non-null   object 
 3   Open                1239 non-null   float64
 4   High                1239 non-null   float64
 5   Low                 1239 non-null   float64
 6   Close               1239 non-null   float64
 7   Adj. Close          1239 non-null   float64
 8   Volume              1239 non-null   int64  
 9   Dividend            1239 non-null   float64
 10  Shares Outstanding  1239 non-null   float64
dtypes: float64(7), int64(2), object(2)
memory usage: 116.2+ KB


In [ ]:
# check which floats can be transformed

In [23]:
company_bruker.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 930 to 930
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Ticker                         1 non-null      object 
 1   SimFinId                       1 non-null      int64  
 2   Company Name                   1 non-null      object 
 3   IndustryId                     1 non-null      float64
 4   ISIN                           1 non-null      object 
 5   End of financial year (month)  1 non-null      float64
 6   Number Employees               1 non-null      float64
 7   Business Summary               1 non-null      object 
 8   Market                         1 non-null      object 
 9   CIK                            1 non-null      float64
 10  Main Currency                  1 non-null      object 
dtypes: float64(4), int64(1), object(6)
memory usage: 96.0+ bytes


In [24]:
company_bruker.head()

,Ticker,SimFinId,Company Name,IndustryId,ISIN,End of financial year (month),Number Employees,Business Summary,Market,CIK,Main Currency
930,BRKR,1253240,BRUKER CORP,106001.0,US1167941087,12.0,7400.0,Bruker is enabling scientists to make breakthr...,us,1109354.0,USD


We see there are some float64 which might be cleaner to format as int64, column headers such as ID's, which can only be full numbers. Let's double check, we'll do this for the whole dataset.

In [33]:
companies[companies["IndustryId"] % 1 != 0]["IndustryId"].unique()

array([nan])

In [35]:
companies[companies["End of financial year (month)"] % 1 != 0]["End of financial year (month)"].unique()

array([nan])

In [32]:
companies[companies["Number Employees"] % 1 != 0]["Number Employees"].unique()

array([nan])

In [30]:
companies[companies["CIK"] % 1 != 0]["CIK"].unique()

array([nan])

Conclusion:
- prices_bruker: date column is object and not date format, will be changed
- company_bruker: we can convert all floats to integer, to have a cleaner dataset


Comment:
There are missing values in the whole dataset. If more companies are added to the model, these need to be handled.

In [22]:
prices_bruker.head()

,Ticker,SimFinId,Date,Open,High,Low,Close,Adj. Close,Volume,Dividend,Shares Outstanding
812641,BRKR,1253240,2019-04-01,38.76,39.01,38.47,38.87,38.20,761845,0.0,156814676.0
812642,BRKR,1253240,2019-04-02,38.94,39.01,38.39,38.53,37.86,666801,0.0,156814676.0
812643,BRKR,1253240,2019-04-03,38.74,39.24,38.63,39.01,38.33,2647355,0.0,156814676.0
812644,BRKR,1253240,2019-04-04,38.95,39.01,37.82,38.32,37.66,503730,0.0,156814676.0
812645,BRKR,1253240,2019-04-05,38.41,39.23,38.30,38.41,37.74,509516,0.0,156814676.0


<a id='5'></a>
## Step 5 - Preparation for merging

In [5]:
prices.head()

,Ticker,SimFinId,Date,Open,High,Low,Close,Adj. Close,Volume,Dividend,Shares Outstanding
0,A,45846,2019-04-01,80.96,81.77,80.96,81.56,78.33,1522681,0.16,317515869.0
1,A,45846,2019-04-02,81.71,81.76,81.03,81.14,77.93,1203037,NaN,317515869.0
2,A,45846,2019-04-03,81.54,82.02,81.46,81.94,78.70,2141025,NaN,317515869.0
3,A,45846,2019-04-04,81.84,82.05,80.44,80.83,77.63,2180112,NaN,317515869.0
4,A,45846,2019-04-05,81.19,81.92,81.05,81.47,78.25,1502875,NaN,317515869.0
